In [ ]:
import sys
import os
build_path = os.path.join(os.getcwd(), "..", "build")
!cmake -B {build_path} -DPYTHON_EXECUTABLE={sys.executable} -Wno-dev ..
!cmake --build {build_path}
if build_path not in sys.path:
    sys.path.append(build_path)
import pypumila
# to reload pypumila, restart python kernel.
# importlib.reload(pypumila) <- does not work

In [ ]:
os.chdir(build_path)
sim = pypumila.GameSim()

In [ ]:
import numpy as np

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
import random

In [ ]:
model = pypumila.Pumila4.make_shared(0.01)
model_first = model.copy()
rewards = []
diffs = []
scores = []
it = 10000
with tqdm(total=it) as pbar:
    while pbar.n < it:
        sim = pypumila.GameSim()
        rnd_p = random.random()
        # print(rnd_p)
        for e in range(50):
            pbar.update()
            a = model.get_action_rnd(sim.field, rnd_p)
            model.learn_step(sim.field)
            # diff = model.mean_diff
            sim.put(pypumila.actions[a])
            while not sim.is_free_phase():
                sim.step()
            reward = pypumila.Pumila2.calc_reward(sim.field)
            rewards.append(reward)
            # diffs.append(diff)
            # このdiffはだいぶ前の学習結果で使ったdiffなので参考にならない
            #print(f"a = {a}, reward = {reward}, chain = {sim.field.prev_chain_num}, diff = {diff}")
        # print("reset sim")
        scores.append(sim.field.total_score)

In [ ]:
model.save_file()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(rewards)

In [ ]:
ar = np.array(rewards)
plt.plot([np.average(ar[i:i+100]) for i in range(len(rewards)-100)])

In [ ]:
plt.plot(diffs)

In [ ]:
plt.plot(scores)

In [ ]:
model = pypumila.Pumila4.make_shared(0.01)
model.load_file()

In [ ]:
sim = pypumila.GameSim()
win = pypumila.Window(sim)
win.step(False, False)

In [ ]:
while True:
    # print(model.main.forward(model.get_in_nodes(sim.field).in_nodes).q)
    a = model.get_action(sim)
    sim.put(pypumila.actions[a])
    while not sim.is_free_phase():
        win.step(True, False)
    for i in range(15):
        win.step(True, False)

In [ ]:
win.quit()